In [ ]:
#Librerías
import pandas as pd
import requests
import os
import patoolib
import datetime
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
import matplotlib.ticker as ticker
import locale

# Locale para Mex
locale.setlocale(locale.LC_ALL, 'es_MX')

In [ ]:
df = pd.DataFrame()
for archivo in archivos_buenfin:
    df_temp = pd.read_csv("profeco/"+archivo[2:6]+"/"+archivo, encoding="utf-8",names=["producto",
                                                                  "presentacion",
                                                                  "marca",
                                                                  "categoria",
                                                                  "catalogo",
                                                                  "precio",
                                                                  "fecharegistro",
                                                                  "cadenacomercial",
                                                                    "giro",
                                                                    "nombrecomercial",
                                                                    "direccion",
                                                                    "estado",
                                                                    "municipio",
                                                                    "latitud",
                                                                    "longitud",
                                                                  ])
    df=pd.concat([df,df_temp],ignore_index=True)

In [ ]:
# Periódos  Buen Fin

periods = ["2015-11-13_2015-11-16",
            "2016-11-18_2016-11-21",
            "2017-11-17_2017-11-20",
            "2018-11-16_2018-11-19",
            "2019-11-15_2019-11-18",
            "2020-11-09_2020-11-20",
            "2021-11-10_2021-11-16",
            "2022-11-18_2022-11-21"]
# Número de semanas
week_numbers = {}
for period in periods:
    start_date, end_date = period.split("_")
    start_week = datetime.datetime.strptime(start_date, "%Y-%m-%d").isocalendar()[1]
    end_week = datetime.datetime.strptime(end_date, "%Y-%m-%d").isocalendar()[1]
    year = int(start_date[:4])
    week_nums = []
    for week in range(start_week-4, end_week+5):
        if week < 1:
            year -= 1
            week_nums.append(52 + week)
        elif week > 52:
            year += 1
            week_nums.append(week - 52)
        else:
            week_nums.append(week)
    if year not in week_numbers:
        week_numbers[year] = []
    week_numbers[year].append(week_nums)

In [ ]:
#Filtrar electrodomésticos
electrodomesticos = df[df["catalogo"]=="ELECTRODOMESTICOS"]
electrodomesticos["fecharegistro"] = pd.to_datetime(electrodomesticos["fecharegistro"], format="%Y-%m-%d")

In [ ]:
#Graficar precio medio de electrodomésticos para cada año. Quiero el registro de cada semana. Utilizar la función elementos_grafica()
for year in electrodomesticos["fecharegistro"].dt.year.unique():
    
    electrodomesticos_year = electrodomesticos[electrodomesticos["fecharegistro"].dt.year==year]
    electrodomesticos_year = electrodomesticos_year.groupby("semana_registro").median()
    electrodomesticos_max = electrodomesticos_year["precio"].max()
    electrodomesticos_min = electrodomesticos_year["precio"].min()
    electrodomesticos_year.plot(y="precio",figsize=(10,5),color="#a1d99b")
    #Gráficar max y min
    #plt.axhline(electrodomesticos_max, color="r", linestyle="--")
    #plt.axhline(electrodomesticos_min, color="r", linestyle="--")
    #media movil de 4 semanas
    electrodomesticos_year["precio"].rolling(2).median().plot(figsize=(10,5),style="r--")
    #Precio máximo
    #Incluir área de Buen Fin con base en la lista periods. Poner el área en gris con transparencia 0.2 y etiquetarla con "Buen Fin"
    for period in periods:
        start_date, end_date = period.split("_")
        start_week = datetime.datetime.strptime(start_date, "%Y-%m-%d").isocalendar()[1]
        end_week = datetime.datetime.strptime(end_date, "%Y-%m-%d").isocalendar()[1]
        if year == int(start_date[:4]):
            plt.axvspan(start_week, end_week, alpha=0.2, color='grey')
            #plt.text(start_week+1, (electrodomesticos_year["precio"].min() + electrodomesticos_year["precio"].max())/2, "Buen Fin", size=8, weight="bold")


    plt.xticks(rotation=90, size=8)
    plt.grid(False)
    plt.title("Precio mediano de electrodomésticos en "+str(year), size=12, weight="bold")
    plt.xlabel("Semana del año")
    plt.ylabel("Precio (MXN)")
    plt.legend(['Precio mediano (MXN)','Media móvil 2 semanas'], frameon=False)
        #Eliminar los bordes
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    #Eje y con separadores de miles
    plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))
    #Añadir fuente
    
    plt.text(0.22, -0.15, "Nota: El área sombreada corresponde al período del Buen Fin de cada año", size=8, ha="center", transform=plt.gca().transAxes)
    
    #Salvar la gráfica
    plt.savefig("profeco/precio_med_elect_sem_"+str(year)+".png", bbox_inches="tight",transparent=True)
